## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [5]:
# clase producto --------------------------------------------------------------- 

class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self._codigo = codigo
        self._nombre = nombre
        self._tipo = tipo
        self.precio = precio
        self.cantidad = cantidad

    @property
    def codigo(self):
        return self._codigo

    @property
    def nombre(self):
        return self._nombre

    @property
    def tipo(self):
        return self._tipo

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, value):
        if 10 <= value <= 10000:
            self._precio = value
        # else:
        #     raise ValueError("Precio debe estar entre 10 y 10000.")

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, value):
        if 0 <= value <= 1000:  
            self._cantidad = value
        # else:
        #     raise ValueError("Cantidad debe estar entre 0 y 1000.")

    def valorTotal(self):
        return self.cantidad * self.precio


In [6]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [7]:

# clase oferta ----------------------------------------------------------------------------------- 

class Oferta:
    def esAplicable(self, producto, cantidad):
        raise NotImplementedError("Este método debe ser implementado en subclases")

    def calcularDescuento(self, producto, cantidad):
        raise NotImplementedError("Este método debe ser implementado en subclases")


class OfertaDescuento:
    def __init__(self, porcentaje, codigos, tipos=None):  
        self.codigos = codigos
        self.tipos = tipos if tipos is not None else [] 

    def esAplicable(self, producto, cantidad):
        return producto.codigo in self.codigos

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            return (producto.precio * cantidad) * (self.porcentaje / 100)
        return 0


class Oferta2x1:
    def __init__(self, tipos, codigos):
        self.tipos = tipos
        self.codigos = codigos

    def esAplicable(self, producto, cantidad):
        return producto.codigo in self.codigos and producto.tipo in self.tipos

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            if cantidad >= 2:
                return (cantidad // 2) * producto.precio
        return 0


In [8]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

AttributeError: 'OfertaDescuento' object has no attribute 'porcentaje'

# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [11]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    def agregar(self, producto):
        self.productos.append(producto)

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(p.cantidad for p in self.productos)

    @property
    def valorTotal(self):
        return sum(p.valorTotal() for p in self.productos)

    def calcularDescuento(self, producto, cantidad):
        descuento = 0
        for oferta in self.ofertas:
            if oferta.es_aplicable(producto, cantidad):
                descuento += oferta.calcularDescuento(producto, cantidad)
        return descuento

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def vender(self, producto, cantidad):
        if producto.cantidad >= cantidad:
            producto.cantidad -= cantidad
        else:
            raise ValueError("No hay suficiente cantidad para vender.")

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    def guardar(self, filename):
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['codigo', 'nombre', 'precio', 'tipo', 'cantidad'])
            for producto in self.productos:
                writer.writerow([producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.cantidad])

    @classmethod
    def leer(cls, filename):
        catalogo = cls() 
        with open(filename, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader)  
            for row in reader:
                codigo, nombre, precio, tipo, cantidad = row
                producto = Producto(codigo, nombre, float(precio), tipo, int(cantidad))
                catalogo.agregar(producto)  
        return catalogo


In [12]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

TypeError: Oferta2x1.__init__() missing 1 required positional argument: 'codigos'

# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [186]:
# cliente------------------------------------------------------------------------- 

import re

class Cliente:
    def __init__(self, nombre, cuit):
        self._nombre = None
        self.nombre = nombre  
        self._cuit = None
        self.cuit = cuit  

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, value):
        if not value or len(value) > 100:
            raise ValueError("El nombre debe tener entre 1 y 100 caracteres.")
        self._nombre = value

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, value):
        if not self.validar_cuit(value):
            raise ValueError("El CUIT debe tener el formato XX-XXXXXXXX-X y ser un número válido.")
        self._cuit = value

    @staticmethod
    def validar_cuit(cuit):
        # Verifica el formato del CUIT
        return bool(re.match(r'^\d{2}-\d{8}-\d{1}$', cuit))



In [ ]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

In [ ]:
# factura---------------------------------------------------------------------------- 

class Factura:
    _ultimo_numero = 0

    @classmethod
    def ultimaFactura(cls, numero):
        cls._ultimo_numero = numero

    @classmethod
    def nuevoNumero(cls):
        cls._ultimo_numero += 1
        return cls._ultimo_numero

    def __init__(self, catalogo, cliente):
        self.numero = self.nuevoNumero()
        self.fecha = self.obtener_fecha()
        self.cliente = cliente
        self.catalogo = catalogo
        self.productos = {}

    @staticmethod
    def obtener_fecha():
        from datetime import datetime
        return datetime.now().strftime('%Y-%m-%d')

    def agregar(self, producto, cantidad):
        if producto.cantidad >= cantidad:
            if producto.codigo in self.productos:
                self.productos[producto.codigo]['cantidad'] += cantidad
            else:
                self.productos[producto.codigo] = {'producto': producto, 'cantidad': cantidad}
            self.catalogo.vender(producto, cantidad)  
        else:
            raise ValueError("No hay suficiente cantidad para agregar.")

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(item['cantidad'] for item in self.productos.values())

    @property
    def subtotal(self):
        total = 0
        for item in self.productos.values():
            producto = item['producto']
            cantidad = item['cantidad']
            total += producto.precio * cantidad
        return total

    @property
    def descuentos(self):
        total_descuentos = 0
        for item in self.productos.values():
            producto = item['producto']
            cantidad = item['cantidad']
            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            total_descuentos += descuento
            print(f"Producto: {producto.nombre}, Cantidad: {cantidad}, Descuento: {descuento}")  # Imprime cada descuento
        return total_descuentos

    @property
    def total(self):
        return self.subtotal - self.descuentos

    def imprimir(self):
        factura_str = f"Factura: {self.numero}\n"
        factura_str += f"Fecha  : {self.fecha}\n"
        factura_str += f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})\n\n"

        for item in self.productos.values():
            producto = item['producto']
            cantidad = item['cantidad']
            subtotal = producto.precio * cantidad
            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            descripcion_oferta = ""
            
            # Descripción de oferta
            if descuento > 0:
                if producto.tipo == 'galleta':
                    descripcion_oferta = "Oferta 2x1"
                else:
                    descripcion_oferta = "Descuento 10%"  

            # Imprimir línea de producto
            factura_str += f"- {cantidad}u {producto.nombre:<30} x ${producto.precio:.2f} = ${subtotal:.2f}\n"
            if descuento > 0:
                factura_str += f"{' ' * 3}{descripcion_oferta:<30} - ${descuento:.2f}\n"

        # Totales
        factura_str += f"\n{' ' * 45}Subtotal:   ${self.subtotal:.2f}\n"
        factura_str += f"{' ' * 45}Descuentos: ${self.descuentos:.2f}\n"
        factura_str += f"{' ' * 45}{'-' * 20}\n"
        factura_str += f"{' ' * 45}Total:      ${self.total:.2f}\n"
        
        return factura_str


In [ ]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

In [ ]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")